In [1]:
import pandas as pd
from pathlib import Path

alex_result_dir = Path(r'\\allen\programs\braintv\workgroups\nc-ophys\visual_behavior\ophys_glm\v_24_events_all_L2_optimize_by_session')
alex_result_df = pd.read_pickle(alex_result_dir / 'results.pkl')
gad2_glm_dir = Path(r'\\allen\programs\braintv\workgroups\nc-ophys\mFish_glm\ophys_glm\v_version_07_events')
gad2_table = pd.read_csv(gad2_glm_dir / 'experiment_table_v_version_07_events.csv')

In [7]:
alex_result_df.columns

Index(['dropout', 'absolute_change_from_full', 'avg_L2_regularization',
       'variance_explained', 'variance_explained_full',
       'avg_cv_var_test_full_comparison_raw', 'avg_cv_var_test_raw',
       'avg_cv_var_test_sem', 'cell_L2_regularization',
       'fraction_change_from_full', 'shuffle_cells', 'shuffle_time',
       'cell_specimen_id', 'adj_fraction_change_from_full', 'adj_dropout_raw',
       'adj_variance_explained', 'adj_variance_explained_full',
       'avg_cv_adjvar_test_full_comparison_raw', 'avg_cv_adjvar_test_raw',
       'cell_roi_id', 'glm_version', 'ophys_experiment_id', 'ophys_session_id',
       'entry_time_utc', 'equipment_name', 'full_genotype', 'mouse_id',
       'reporter_line', 'driver_line', 'sex', 'age_in_days', 'cre_line',
       'indicator', 'session_number', 'prior_exposures_to_session_type',
       'prior_exposures_to_image_set', 'prior_exposures_to_omissions',
       'behavior_session_id', 'ophys_container_id', 'project_code',
       'imaging_depth',

In [25]:
vbp_table = alex_result_df.query('equipment_name == "MESO.1"')[['mouse_id', 'cre_line', 'session_type','ophys_session_id']].drop_duplicates()
gad2_table_simplified = gad2_table[['mouse_id', 'cre_line', 'session_type','ophys_session_id']].drop_duplicates()
all_table = pd.concat([vbp_table, gad2_table_simplified], axis=0)

In [26]:
len(gad2_table_simplified)

147

In [27]:
all_table

,mouse_id,cre_line,session_type,ophys_session_id
15812,451787,Slc17a7-IRES2-Cre,OPHYS_2_images_A_passive,886367984
24524,456915,Slc17a7-IRES2-Cre,OPHYS_3_images_A,904771513
25964,457841,Sst-IRES-Cre,OPHYS_1_images_A,951410079
28124,457841,Sst-IRES-Cre,OPHYS_2_images_A_passive,952430817
30384,457841,Sst-IRES-Cre,OPHYS_4_images_B,957020350
...,...,...,...,...
1135,646883,Gad2-IRES-Cre,TRAINING_4_images_A_training,1230663936
1143,646883,Gad2-IRES-Cre,TRAINING_4_images_A_training,1230890037
1151,646883,Gad2-IRES-Cre,TRAINING_4_images_A_training,1231139212
1159,646883,Gad2-IRES-Cre,TRAINING_4_images_A_training,1231421272


In [29]:
vbp_table_oeid = alex_result_df.query('equipment_name == "MESO.1"')[['mouse_id', 'cre_line', 'session_type','ophys_session_id', 'ophys_experiment_id']].drop_duplicates()
gad2_table_simplified_oeid = gad2_table[['mouse_id', 'cre_line', 'session_type','ophys_session_id', 'ophys_experiment_id']].drop_duplicates()
all_table_oeid = pd.concat([vbp_table_oeid, gad2_table_simplified_oeid], axis=0)


In [41]:
base_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\boosting')
selected_oeids = pd.read_csv(base_dir / 'boosting_test_oeids.csv', index_col=0)


In [42]:
priority_osid = []
for oeid in selected_oeids.index.values:
    priority_osid.append(all_table_oeid.query('ophys_experiment_id == @oeid')['ophys_session_id'].values[0])

In [44]:
all_table.priority = False

In [45]:
for osid in priority_osid:
    all_table.loc[all_table.ophys_session_id == osid, 'priority'] = True

In [46]:
save_dir  = Path(r'\\allen\programs\mindscope\workgroups\learning')
save_fn = 'code_ocean_upload_vbp_gad2_SAC2023.csv'
all_table.to_csv(save_dir / save_fn, index=False)